In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

#Получаем данные со странице, по указанной ссылке
r = requests.get("https://mfd.ru/marketdata/?id=5&group=16&mode=3&sortHeader=name&sortOrder=1&selectedDate=01.11.2019")
#преобразуем её в объект
html = BeautifulSoup(r.content)
#получаем данные из тега "table", id  у которого имеет значение "marketDatalist"
table = html.find('table', {'id': 'marketDataList'})
#создаём список где будем хранить данные строк
rows = []
#получаем все данные которые имеют тег 'tr' 
table_rows = table.find_all('tr')
#обходим циклом полученные данные, из каждой строки (тега "td")
#получаем содержимое в виде текста
for table_row in table_rows:
    #в строку (table_row), с помощью генератора осуществляем добавления текста 
    #из ячеек (table_data), table_data обалвяем данные из содержимого с тегом 'td'
    #с помощью параметра strip=True убираем переносы строк и пробелы 
    table_row = [table_data.get_text(strip=True) for table_data in table_row.find_all('td')]
    #если длина строки больше нуля
    if len(table_row) > 0:
        #мы в список строк, добавляем полученную нами из table_rows строку
        rows.append(table_row)
#создаём фрейм данных куда добавляем наш список "rows", присваиваем нудные нам имена для колонок
data = pd.DataFrame(rows, columns=["Тикер", "Дата", "Сделки", "рост", "% сделок", "Цена-З.","Цена-О.", "Мин", "Макс", "Ср", "шт", "руб","Всего"])
#15)	в созданный фрейм, из серии данных "Сделки" добавляем только те значения, которые не равны "N/A"
data = data[data["Сделки"] != "N/A"]
#из столбца "процент сделок" все символы "дефис" заменяем на символ "минус"
#так их наличие как они затрудняют сортировку
#в получившихся строк знак процента и заменяем на пустое значения
#полученные данные преобразуем к типу float
data["% сделок"] = data["% сделок"].str.replace("−", "-").str.replace("%","").astype(float)
#в данном фрейме данных из столбца "% сделок" формируем индекс
data = data.set_index("% сделок")
#сортируем данные по индексу, в порядке убывания (ascending=False) 
data = data.sort_index(ascending=False)
#выводим первую позицию, из серии данных "Тикер", это и является ответом
print(data["Тикер"].head(1))
    
